## Python `async` Usage

| Syntax            | Purpose                          | Usage                                  |
|-------------------|----------------------------------|----------------------------------------|
| **`async def`**   | Define async/coroutine functions | `async def my_func():`                 |
| - **`async for`** | Iterate over async iterators     | `async for item in async_generator():` |
| - **`await`**     | Wait for **coroutine** objects   | `result = await some_coroutine()`      |
| **`async with`**  | Async context managers           | `async with async_resource():`         |

## The `async` syntax Rule

| Function Pattern       | How to Consume              | What You Get       |
|------------------------|-----------------------------|--------------------|
| `async def` + `yield`  | `async for` item in func(): | Each yielded item  |
| `async def` + `return` | result = `await` func()     | The returned value |

`async def` with both `yield` and `return`

- Any yield → AsyncGenerator → use async for (HIGH priority)
- No yield → Coroutine → use await

In [3]:
### `async def` + `yield`

# Pattern 1: yield -> use async for
async def get_users():
    yield user1
    yield user2
    yield user3
    # same as `return None`


# ✅ Correct consumption
async def my_function():
    # `for user in await get_users():` -  ❌ Wrong - You may expect to see the keyword `await`
    # but in fact - ✅ `async for` does the "await" in an iteration format
    async for user in get_users():
        process(user)


# ❌ Wrong - can't await an AsyncGenerator
##### result = await get_users()  # TypeError

In [7]:
### `async def` + `return`

# Pattern 2: return -> use await  
async def get_user():
    return single_user


# ✅ Correct consumption
async def my_function():
    user = await get_user()


# ❌ Wrong - can't iterate a Coroutine
##### async for u in get_user():  # TypeError
      

In [14]:
! pip install sqlalchemy

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [16]:
from typing import AsyncGenerator
from sqlalchemy.ext.asyncio import AsyncSession
## async with

async def get_session() -> AsyncGenerator[AsyncSession, None]:  # 3 ✅
    # db_local_session = async_sessionmaker(...)
    async with Database.db_local_session() as session:  # 1 ✅
        try:
            yield session  # 4a ✅ `yield` relates to `async def`, the caller needs to use `async for`
            await session.commit()  # 2 ✅
        except Exception:
            await session.rollback()
            raise


# ✅ Correct usage - async context manager
@asynccontextmanager
async def get_db_session() -> AsyncIterator[AsyncSession]:
    async for session in Database.get_session():  # 4b ✅
        yield session


ModuleNotFoundError: No module named 'sqlalchemy'

### The sequence:

1. ✅ `async with` needed because `Database.db_local_session()` **returns** an async context manager
2. ✅ `await` needed because `session.commit()` is an async function
1. ✅ `async def` needed because we use `async with` keyword inside
3. ✅ `async def` needed because we use `await` keyword inside
4. ✅ `async def` + `yield` (4a) needs `async for` (4b)

### Async Context Manager - means it has `__aenter__` and `__aexit__`

```python
# Database.session() returns an object that has:
class AsyncSession:
    async def __aenter__(self): ...  # ← async context manager

    async def __aexit__(self): ...  # ← async context manager
```

### `await` has nothing to do with `async with`

| Keyword      | Why Needed                                                  |
|--------------|-------------------------------------------------------------|
| `await`      | `session.commit()` is async                                 |
| `async with` | `Database.db_local_session()` returns async context manager |
